In [ ]:
#  Pre-Process Retrosheet Data.ipynb - Pre-processes data from retrosheet.org
#     Copyright (C) 2020  Geoffrey G. Messier
# 
#     This program is free software: you can redistribute it and/or modify
#     it under the terms of the GNU General Public License as published by
#     the Free Software Foundation, either version 3 of the License, or
#     (at your option) any later version.
# 
#     This program is distributed in the hope that it will be useful,
#     but WITHOUT ANY WARRANTY; without even the implied warranty of
#     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#     GNU General Public License for more details.
# 
#     You should have received a copy of the GNU General Public License
#     along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [1]:
%matplotlib inline
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import scipy as sci
import scipy.special as scisp
import scipy.stats as scist
import datetime, copy, imp
import re
import sys


import MySQLdb

import pymysql.cursors;

from tqdm import tqdm
from mpl_toolkits.mplot3d import Axes3D

tqdm.pandas()
plt.ion()

### Introduction
I used this notebook to create the `Games` table in the practice SQL database using data downloaded from retrosheet.org.  Much of this work involved extracting the different game events that I wanted to isolate for further analysis.  The final output of this notebook is an HDF and CSV export of `Games`.

### SQL Database Connection
I have the original retrosheet data on a private SQL server running locally on my computer.  The first step is to extract the relevant columns from the retrosheet `events` table which are generated from their events files.  For this, I use the MySQLdb library and the SQL query capability of pandas.

The goal is to create a baseball player data record where time stamped events are created for each player.  This data is similar to a shelter, justice or healthcare record where a client/patient experiences a series of events.  The analysis is all about individual journey mapping and trying to identify events that predict outcomes.  Journey mapping could include different visualization techniques.  The prediction work emphasizes making decisions early in a player's timeline and producing prediction rules that are human recognizable.

In [2]:
# -- Conect to Database --
connection = MySQLdb.connect(host='127.0.0.1',
                         user='bbos',
                         password='bbos',
                         db='retrosheet');

In [3]:
commandStr = """
    select
        GAME_ID GameId,
        AWAY_TEAM_ID AwayTeamId,
        INN_CT Inning,
        BAT_HOME_ID TopBottom,
        PITCH_SEQ_TX PitchSeq,
        EVENT_TX Result,
        BAT_ID Batter,
        BASE1_RUN_ID RunnerBase1,
        BASE2_RUN_ID RunnerBase2,
        BASE3_RUN_ID RunnerBase3,
        POS2_FLD_ID Catcher,
        POS3_FLD_ID FirstBase,
        POS4_FLD_ID SecondBase,
        POS5_FLD_ID ThirdBase,
        POS6_FLD_ID ShortStop,
        POS7_FLD_ID LeftField,
        POS8_FLD_ID CenterField,
        POS9_FLD_ID RightField
    from events
    where events.GAME_ID regexp '^...2018'
    """

df = pd.read_sql(commandStr,connection)
df

,GameId,AwayTeamId,Inning,TopBottom,PitchSeq,Result,Batter,RunnerBase1,RunnerBase2,RunnerBase3,Catcher,FirstBase,SecondBase,ThirdBase,ShortStop,LeftField,CenterField,RightField
0,ANA201804020,CLE,1,0,BCBSX,8/L+,lindf001,,,,maldm001,martj007,cozaz001,valbl001,simma001,uptoj001,troum001,calhk001
1,ANA201804020,CLE,1,0,CBCX,31/G,kipnj001,,,,maldm001,martj007,cozaz001,valbl001,simma001,uptoj001,troum001,calhk001
2,ANA201804020,CLE,1,0,CBSFFX,31/G,ramij003,,,,maldm001,martj007,cozaz001,valbl001,simma001,uptoj001,troum001,calhk001
3,ANA201804020,CLE,1,1,X,8/F,cozaz001,,,,gomey001,alony001,kipnj001,ramij003,lindf001,naqut001,zimmb001,chisl001
4,ANA201804020,CLE,1,1,CCT,K,troum001,,,,gomey001,alony001,kipnj001,ramij003,lindf001,naqut001,zimmb001,chisl001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191046,WAS201809260,MIA,7,0,FFBX,53/G,dietd001,,,,kiebs001,reynm001,sanca007,renda001,turnt001,sotoj001,roblv001,harpb003
191047,WAS201809260,MIA,7,1,.BBFBB,W,kiebs001,,,,wallc001,dietd001,bostc001,andeb006,rojam002,orter001,brinl001,obrip002
191048,WAS201809260,MIA,7,1,.SFS,K,eatoa002,kiebs001,,,wallc001,dietd001,bostc001,andeb006,rojam002,orter001,brinl001,obrip002
191049,WAS201809260,MIA,7,1,CSFX,6/P,roblv001,kiebs001,,,wallc001,dietd001,bostc001,andeb006,rojam002,orter001,brinl001,obrip002


#### Games Table Format
- Much of the code below involves analyzing the letters in the retrosheet event strings.  These are described in detail [here](https://www.retrosheet.org/eventfile.htm).
- Any irrelevant fields for a row (ie. strike count for a defensive play entry) are null.
- Pitchers are excluded from the analysis.
- Fields:
  - `Time`: A timestamp that starts at 12pm (first game), 3:05pm (second game), 6:10 (third game).  Increments by 6 minutes for every at bat (based on average 3.5 batters/inning and 20min innings).
  - `GameId`
  - `Inning`
  - `TopBottom`
  - `PlayerId`
  - `Position`: Batter, Catcher, FirstBase, SecondBase, ThirdBase, ShortStop, LeftField, CenterField, RightField
  - `Event`: 
   - Event values for Batter: StrikeOut, PutOut (out because of a defensive play), Walk, Hit, Run, Steal 
   - Event values for defensive positions: Out, Error (player assigned an error)
  - Fields associated with Batter:
    - `PitchCount`: Number of pitches thrown during the at bat.
    - `Rbi`: Number of runners that score as a result of a player's hit.
    - `StartBase`: 1 - 4 where home base is 4.
    - `EndBase`
  - Fields associated with defensive positions:
    - `Outs`: Number of outs resulting from the defensive play.
    - `Players`: Number of players involved (one if the player got the out by himself).

In [4]:
topBottom = { 0: 'Top', 1: 'Bottom' }
playerTeam = { }
fielders = { 
    2: 'Catcher', 3: 'FirstBase', 4: 'SecondBase', 5: 'ThirdBase', 
    6: 'ShortStop', 7: 'LeftField', 8: 'CenterField', 9: 'RightField'
}


def CreatePlayerEvent(eventType, position, playerId, teamId,
                      pitchCount = np.nan, rbi = np.nan, startBase = np.nan, endBase = np.nan,
                      outs = np.nan, players = np.nan, score = [ 0, 0 ]):
     return pd.Series({ 
        'HomeScore': score[0],'AwayScore': score[1],
        'PlayerId': playerId,'Position': position, 
        'PlayerTeam': teamId,'Event': eventType, 
        'PitchCount': pitchCount,'Rbi': rbi,
        'StartBase': startBase,'EndBase': endBase,
        'Outs': outs,'Players': players
        })
            



In [5]:
hitRegex = re.compile("^(S\d|D\d|T\d|DGR|HR|HP)"); # For now, includes being hit by a pitch.
runRegex = re.compile("\d-(?:\d|H)")
stealRegex = re.compile("SB(?:\d|H)")
singleRegex = re.compile("^S|^HP");
doubleRegex = re.compile("^D");
tripleRegex = re.compile("^T");
homerunRegex = re.compile("^H");
rbiRegex = re.compile("-H");
walkRegex = re.compile("^(?:IW|W(?:[^P]|$))")
strikeOutRegex = re.compile("^K")
errorRegex = re.compile("E\d")
pitchRegex = re.compile("(B|C|F|H|I|K|L|M|N|O|P|Q|R|S|T|U|V|X|Y)")
fieldOutRegex = re.compile("^(?:\d+(?:\(.\))*)+")


In [6]:
def ProcessGame(atBats):
    
    events = pd.DataFrame(
        columns = [ 
            'Time', 'HomeTeam', 'AwayTeam', 'Inning', 'TopBottom', 'HomeScore', 'AwayScore', 
            'PlayerId', 'Position', 'PlayerTeam', 
            'Event', 'PitchCount', 'Rbi', 'StartBase', 'EndBase', 'Outs', 'Players' ])
    
    gameStr = atBats.iloc[0].GameId
    dateStr = gameStr[3:7] + '-' + gameStr[7:9] + '-' + gameStr[9:11]
        
    if gameStr[-1] == '0':
        curTime = pd.to_datetime(dateStr + ' 12:00')
    elif gameStr[-1] == '1':
        curTime = pd.to_datetime(dateStr + ' 15:00')
    elif gameStr[-1] == '2':
        curTime = pd.to_datetime(dateStr + ' 18:00')
    else:
        sys.exit("ERROR: Can't handle more than a triple header.")
        
    nEvents = 0
    score = [ 0, 0 ]
    teams = [ gameStr[0:3], atBats.iloc[0].AwayTeamId ]
    
    for ind,atBat in atBats.iterrows():
    
        topBottomId = int(atBat.TopBottom)
        inFieldId = topBottomId
        atBatId = (inFieldId+1) % 2
        
        timeInfo = pd.Series({'Time': curTime, 
                              'HomeTeam': teams[0],
                              'AwayTeam': teams[1],
                              'Inning': int(atBat.Inning), 
                              'TopBottom': topBottom[topBottomId]
                             })
    
        pitchCount = len(pitchRegex.findall(atBat.PitchSeq))
                
        # --------- Fielders' Out -----------
        if fieldOutRegex.search(atBat.Result):
            
            plays = re.split('\)',fieldOutRegex.findall(atBat.Result)[0])
            plays = list(filter(None,plays))
            nOuts = len(plays)
            
            nFielders = 0
            for play in plays:
                players = re.split('\(',play)
                nFielders += len(players[0])
            
            for play in plays:
                
                players = re.split('\(',play)
                
                if (len(plays) == 1      # Batter is put out if there's no double/triple play
                    or players[0] == '3' # Final fielder is first base
                    or 'B' in play):     # Batter is explicitly out
                        
                    eventInfo = CreatePlayerEvent(
                        'PutOut', 
                        position = 'Batter',
                        playerId = atBat.Batter, 
                        teamId = teams[atBatId],
                        pitchCount = pitchCount,
                        score = score)
            
                    events.at[nEvents] = pd.concat([ timeInfo, eventInfo ])
                    nEvents += 1
            
                for fielder in list(players[0]):
                
                    posNum = int(fielder)
                
                    if posNum == 1: # Ignore pitchers
                        continue
                    
                    eventInfo = CreatePlayerEvent(
                        'Out', 
                        position = fielders[posNum],
                        playerId = atBat[fielders[posNum]], 
                        teamId = teams[inFieldId],
                        outs = nOuts,
                        players = nFielders,
                        score = score)
            
                    events.at[nEvents] = pd.concat([ timeInfo, eventInfo ])
                    nEvents += 1        

                    
        # --------- Hit -----------
        if hitRegex.search(atBat.Result):

            if singleRegex.search(atBat.Result):
                endBase = 1
            elif doubleRegex.search(atBat.Result):
                endBase = 2
            elif tripleRegex.search(atBat.Result):
                endBase = 3
            else:
                endBase = 4
                score[atBatId] += 1
            
            eventInfo = CreatePlayerEvent(
                'Hit',
                position = 'Batter',
                playerId = atBat.Batter,
                teamId = teams[atBatId],
                pitchCount = pitchCount,
                rbi = len(rbiRegex.findall(atBat.Result)), 
                endBase = endBase,
                score = score)
            
            events.at[nEvents] = pd.concat([ timeInfo, eventInfo ])
            nEvents += 1

            
        # --------- Walk -----------
        if walkRegex.search(atBat.Result):
            
            eventInfo = CreatePlayerEvent(
                'Walk',
                position = 'Batter',
                playerId = atBat.Batter,
                teamId = teams[atBatId],
                pitchCount = pitchCount,
                endBase = 1,
                score = score)

            events.at[nEvents] = pd.concat([ timeInfo, eventInfo ])
            nEvents += 1

            
        # --------- Run -----------
        if runRegex.search(atBat.Result):
            
            for runStr in runRegex.findall(atBat.Result):

                startBase = int(runStr[0])
                if runStr[-1] == 'H':
                    endBase = 4
                    score[atBatId] += 1
                else:
                    endBase = int(runStr[-1])
            
                eventInfo = CreatePlayerEvent(
                    'Run',
                    position = 'Runner',
                    playerId = atBat['RunnerBase%d' % startBase],
                    teamId = teams[atBatId],
                    startBase = startBase,
                    endBase = endBase,
                    score = score)

                events.at[nEvents] = pd.concat([ timeInfo, eventInfo ])
                nEvents += 1

                
        # --------- Strike Out -----------
        if strikeOutRegex.search(atBat.Result):
            
            eventInfo = CreatePlayerEvent(
                'StrikeOut',
                position = 'Batter',
                playerId = atBat.Batter,
                teamId = teams[atBatId],
                pitchCount = pitchCount,
                score = score)
            
            events.at[nEvents] = pd.concat([ timeInfo, eventInfo ])
            nEvents += 1

    
        # --------- Steal -----------
        if stealRegex.search(atBat.Result):
            
            for stealStr in stealRegex.findall(atBat.Result):

                if stealStr[-1] == 'H':
                    endBase = 4
                    score[atBatId] += 1
                else:
                    endBase = int(stealStr[-1])
                startBase = endBase-1
            
                eventInfo = CreatePlayerEvent(
                    'Steal',
                    position = 'Runner',
                    playerId = atBat['RunnerBase%d' % startBase],
                    teamId = teams[atBatId],
                    startBase = startBase,
                    endBase = endBase,
                    score = score)

                events.at[nEvents] = pd.concat([ timeInfo, eventInfo ])
                nEvents += 1
            
            
        # --------- Error -----------
        if errorRegex.search(atBat.Result):
            
            for errorStr in errorRegex.findall(atBat.Result):

                posNum = int(errorStr[-1])
                
                if posNum == 1:
                    continue

                eventInfo = CreatePlayerEvent(
                    'Error', 
                    position = fielders[posNum],
                    playerId = atBat[fielders[posNum]], 
                    teamId = teams[inFieldId],
                    score = score)
                
                events.at[nEvents] = pd.concat([ timeInfo, eventInfo ])
                nEvents += 1
                
                
        curTime += pd.Timedelta('6 min')

    return events
    
#events = ProcessGame(df)  

In [7]:
events = df.groupby('GameId').progress_apply(ProcessGame)

100%|██████████| 2431/2431 [10:45<00:00,  3.77it/s]


In [8]:
events

Time HomeTeam AwayTeam Inning TopBottom  \
GameId                                                                     
ANA201804020 0    2018-04-02 12:00:00      ANA      CLE      1       Top   
             1    2018-04-02 12:00:00      ANA      CLE      1       Top   
             2    2018-04-02 12:06:00      ANA      CLE      1       Top   
             3    2018-04-02 12:06:00      ANA      CLE      1       Top   
             4    2018-04-02 12:12:00      ANA      CLE      1       Top   
...                               ...      ...      ...    ...       ...   
WAS201809260 126  2018-09-26 18:48:00      WAS      MIA      7    Bottom   
             127  2018-09-26 18:48:00      WAS      MIA      7    Bottom   
             128  2018-09-26 18:54:00      WAS      MIA      7    Bottom   
             129  2018-09-26 18:54:00      WAS      MIA      7    Bottom   
             130  2018-09-26 18:54:00      WAS      MIA      7    Bottom   

                 HomeScore AwayScore  PlayerId     Position PlayerTeam  \
GameId                                                                   
ANA201804020 0           0         0  lindf001       Batter        CLE   
             1           0         0  troum001  CenterField        ANA   
             2           0         0  kipnj001       Batter        CLE   
             3           0         0  martj007    FirstBase        ANA   
             4           0         0  ramij003       Batter        CLE   
...                    ...       ...       ...          ...        ...   
WAS201809260 126         8         3  roblv001       Batter        WAS   
             127         8         3  rojam002    ShortStop        MIA   
             128         8         3  turnt001       Batter        WAS   
             129         8         3  rojam002    ShortStop        MIA   
             130         8         3  bostc001   SecondBase        MIA   

                   Event PitchCount  Rbi StartBase EndBase Outs Players  
GameId                                                                   
ANA201804020 0    PutOut          5  NaN       NaN     NaN  NaN     NaN  
             1       Out        NaN  NaN       NaN     NaN    1       1  
             2    PutOut          4  NaN       NaN     NaN  NaN     NaN  
             3       Out        NaN  NaN       NaN     NaN    1       2  
             4    PutOut          6  NaN       NaN     NaN  NaN     NaN  
...                  ...        ...  ...       ...     ...  ...     ...  
WAS201809260 126  PutOut          4  NaN       NaN     NaN  NaN     NaN  
             127     Out        NaN  NaN       NaN     NaN    1       1  
             128  PutOut          3  NaN       NaN     NaN  NaN     NaN  
             129     Out        NaN  NaN       NaN     NaN    1       2  
             130     Out        NaN  NaN       NaN     NaN    1       2  

[349310 rows x 17 columns]

In [9]:
events.Time = pd.to_datetime(events.Time)
events.Inning = events.Inning.astype('int')
events.HomeScore = events.HomeScore.astype('int')
events.AwayScore = events.AwayScore.astype('int')
events.PitchCount = events.PitchCount.astype('float')
events.Rbi = events.Rbi.astype('float')
events.StartBase = events.StartBase.astype('float')
events.EndBase = events.EndBase.astype('float')
events.Outs = events.Outs.astype('float')
events.Players = events.Players.astype('float')

In [10]:
events.dtypes

Time          datetime64[ns]
HomeTeam              object
AwayTeam              object
Inning                 int64
TopBottom             object
HomeScore              int64
AwayScore              int64
PlayerId              object
Position              object
PlayerTeam            object
Event                 object
PitchCount           float64
Rbi                  float64
StartBase            float64
EndBase              float64
Outs                 float64
Players              float64
dtype: object

In [14]:
events.to_csv('~/data/baseball/Season-2018.csv',index_label=False,index=False)

In [12]:
events.to_hdf('~/data/baseball/Season-2018.hd5',key='Data')